# 2025 DL Lab8: RL Assignment_Super Mario World

**Your Answer:**    
Hi I'm XXX, XXXXXXXXXX.

## Overview
This project implements a **Deep Reinforcement Learning** pipeline to train an autonomous agent for Super Mario World. Leveraging the **Proximal Policy Optimization (PPO)** algorithm, the system interacts with the **stable-retro** environment to master the YoshiIsland1 level. Key components include a custom Vision Backbone for extracting features from raw pixel data and a suite of Environment Wrappers that handle frame preprocessing, action discretization, and reward shaping to facilitate efficient learning.

Reward function implement  
should do something in the beginning (monster attack)  
Custom PPO implement  
pre train weight 差不多，主要是 reward function  
model weight capacity 1GB  
class name 不要動 (可以新增，但是原本有的不要動)

## Imports

In [1]:
import os
import numpy as np
import retro
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
from stable_baselines3.common.vec_env import VecNormalize

from eval import evaluate_policy, record_video
from custom_policy import VisionBackbonePolicy, CustomPPO

/home/m314834001/Lab8/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Configuration

In [2]:
# Game Settings
GAME = "SuperMarioWorld-Snes"
STATE = "YoshiIsland1"

# Training Settings
TOTAL_STEPS = 13_107_200
TRAIN_CHUNK =    327_680
N_ENVS = 16
LEARNING_RATE = 1e-4

# Evaluation & Recording Settings
EVAL_EPISODES = 3
EVAL_MAX_STEPS = 18000
RECORD_STEPS = 1800

# Directories
LOG_DIR = "./runs_smw"
VIDEO_DIR       = os.path.join(LOG_DIR, "videos")
CKPT_DIR        = os.path.join(LOG_DIR, "checkpoints")
TENSORBOARD_LOG = os.path.join(LOG_DIR, "tb")

os.makedirs(LOG_DIR,   exist_ok=True)
os.makedirs(CKPT_DIR,  exist_ok=True)
os.makedirs(VIDEO_DIR, exist_ok=True)

## Environment Functions

In [4]:
from wrappers import make_base_env
def _make_env_thunk(game: str, state: str):
    """Return a function that creates an environment (for multiprocessing)."""
    def _thunk():
        return make_base_env(game, state)
    return _thunk

def make_vec_env(game: str, state: str, n_envs: int, use_subproc: bool = True):
    """Create a vectorized environment (multiple envs running in parallel)."""
    env_fns = [_make_env_thunk(game, state) for _ in range(n_envs)]
    
    if use_subproc and n_envs > 1:
        vec_env = SubprocVecEnv(env_fns)
    else:
        vec_env = DummyVecEnv(env_fns)

    return vec_env

## Initialize Env & Model

In [5]:
# 1. Create Training Environment
train_env = make_vec_env(GAME, STATE, n_envs=N_ENVS)
# train_env = VecNormalize(train_env, norm_obs=True, norm_reward=True, clip_obs=10., clip_reward=10.)
print(f"Environment created: {GAME} - {STATE} with {N_ENVS} parallel envs.")

checkpoint_path = "None"
# checkpoint_path = "runs_smw/checkpoints/SF84_step_1600000.zip"
# checkpoint_path = "runs_smw/checkpoints/SF84G_6553600.zip"
checkpoint_path = "runs_smw/checkpoints/SF84G_PT_29.zip"

# 2. Initialize Model
if os.path.exists(checkpoint_path):
    print(f"Loading model from {checkpoint_path}...")
    # 讀取現有模型
    model = CustomPPO.load(
        checkpoint_path, 
        env=train_env,
        device="cuda:0" # 確保使用 GPU
    )
else:
    print(f"Fail to load {checkpoint_path}...")
    model = CustomPPO(
        VisionBackbonePolicy,
        train_env,
        policy_kwargs   = dict(normalize_images=False),
        n_epochs        = 4,
        n_steps         = 512,
        batch_size      = 512,
        learning_rate   = LEARNING_RATE,
        verbose         = 1,
        gamma           = 0.99,
        kl_coef         = 1,
        clip_range      = 0.125,
        tensorboard_log = TENSORBOARD_LOG,
    )

Environment created: SuperMarioWorld-Snes - YoshiIsland1 with 16 parallel envs.
Loading model from runs_smw/checkpoints/SF84G_PT_29.zip...


In [7]:
model.save("policy")

## Training Loop

In [6]:
best_mean = -1e18
trained = 9502720
round_idx = 0

try:
    while trained < TOTAL_STEPS:
        round_idx += 1
        chunk = min(TRAIN_CHUNK, TOTAL_STEPS - trained)
        # chunk = 2000

        print(f"\n=== Round {round_idx} | Learn {chunk} steps (Total trained: {trained}) ===")
        
        # --- Train ---
        model.learn(total_timesteps=chunk, reset_num_timesteps=False, tb_log_name='SF84G_1220_20')
        trained += chunk

        # --- Save Checkpoint ---
        ckpt_path = os.path.join(CKPT_DIR, f"SF84G_PT_{int(trained/TRAIN_CHUNK)}.zip")
        model.save(ckpt_path)
        print(f"Saved checkpoint: {ckpt_path}")

        # --- Evaluate ---
        mean_ret, best_ret = evaluate_policy(
            model,
            GAME,
            STATE,
            n_episodes=EVAL_EPISODES,
            max_steps=EVAL_MAX_STEPS,
        )
        print(f"[EVAL] Mean Return: {mean_ret:.3f}, Best Return: {best_ret:.3f}")

        # --- Save Best Model ---
        if mean_ret > best_mean:
            best_mean = mean_ret
            best_path = os.path.join(LOG_DIR, "best_model_A.zip")
            model.save(best_path)
            print(f"New best record. Saved to {best_path}")

        # --- Record Video ---
        record_video(
            model,
            GAME,
            STATE,
            VIDEO_DIR,
            video_len=RECORD_STEPS,
            prefix=f"step_{trained}_mean_{mean_ret:.2f}",
        )

except KeyboardInterrupt:
    print("\nTraining interrupted manually.")

finally:
    train_env.close()
    print("Training finished. Environment closed.")
    
"""
tensorboard --logdir=./runs_smw/tb
"""


=== Round 1 | Learn 327680 steps (Total trained: 9502720) ===
Logging to ./runs_smw/tb/SF84G_1220_20_0


/home/m314834001/Lab8/.venv/lib/python3.12/site-packages/rich/live.py:256: UserWarning: install "ipywidgets" for 
Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

--------------------------------
| time/              |         |
|    fps             | 740     |
|    iterations      | 1       |
|    time_elapsed    | 11      |
|    total_timesteps | 9510912 |
--------------------------------
---------------------------------------
| time/                 |             |
|    fps                | 628         |
|    iterations         | 2           |
|    time_elapsed       | 26          |
|    total_timesteps    | 9519104     |
| train/                |             |
|    approx_kl          | 0.027881095 |
|    entropy_loss       | -1.88       |
|    explained_variance | 0.838       |
|    learning_rate      | 0.0002      |
|    loss               | 0.127       |
|    mean_step_reward   | 0.10536369  |
|    n_updates          | 4644        |
|    policyGradLoss     | -0.00657    |
|    value_loss         | 0.527       |
---------------------------------------
---------------------------------------
| time/                 |             |
|    fps 

Saved checkpoint: ./runs_smw/checkpoints/SF84G_PT_30.zip
[EVAL] Mean Return: 182.850, Best Return: 182.858
New best record. Saved to ./runs_smw/best_model_A.zip
Saved video to ./runs_smw/videos/step_9830400_mean_182.85.mp4

=== Round 2 | Learn 327680 steps (Total trained: 9830400) ===
Logging to ./runs_smw/tb/SF84G_1220_20_0
--------------------------------
| time/              |         |
|    fps             | 735     |
|    iterations      | 1       |
|    time_elapsed    | 11      |
|    total_timesteps | 9838592 |
--------------------------------
---------------------------------------
| time/                 |             |
|    fps                | 622         |
|    iterations         | 2           |
|    time_elapsed       | 26          |
|    total_timesteps    | 9846784     |
| train/                |             |
|    approx_kl          | 0.021215059 |
|    entropy_loss       | -1.87       |
|    explained_variance | 0.972       |
|    learning_rate      | 0.0002      |
| 

Saved checkpoint: ./runs_smw/checkpoints/SF84G_PT_31.zip
[EVAL] Mean Return: 180.600, Best Return: 180.608
Saved video to ./runs_smw/videos/step_10158080_mean_180.60.mp4

=== Round 3 | Learn 327680 steps (Total trained: 10158080) ===
Logging to ./runs_smw/tb/SF84G_1220_20_0
---------------------------------
| time/              |          |
|    fps             | 731      |
|    iterations      | 1        |
|    time_elapsed    | 11       |
|    total_timesteps | 10166272 |
---------------------------------
---------------------------------------
| time/                 |             |
|    fps                | 634         |
|    iterations         | 2           |
|    time_elapsed       | 25          |
|    total_timesteps    | 10174464    |
| train/                |             |
|    approx_kl          | 0.022227086 |
|    entropy_loss       | -1.85       |
|    explained_variance | 0.966       |
|    learning_rate      | 0.0002      |
|    loss               | 0.0215      |
|    me

...

Saved checkpoint: ./runs_smw/checkpoints/SF84G_PT_37.zip
[EVAL] Mean Return: 184.840, Best Return: 184.848
Saved video to ./runs_smw/videos/step_12124160_mean_184.84.mp4

=== Round 9 | Learn 327680 steps (Total trained: 12124160) ===
Logging to ./runs_smw/tb/SF84G_1220_20_0
---------------------------------
| time/              |          |
|    fps             | 741      |
|    iterations      | 1        |
|    time_elapsed    | 11       |
|    total_timesteps | 12132352 |
---------------------------------
---------------------------------------
| time/                 |             |
|    fps                | 630         |
|    iterations         | 2           |
|    time_elapsed       | 26          |
|    total_timesteps    | 12140544    |
| train/                |             |
|    approx_kl          | 0.020190623 |
|    entropy_loss       | -1.82       |
|    explained_variance | 0.993       |
|    learning_rate      | 0.0002      |
|    loss               | -0.0265     |
|    me

Saved checkpoint: ./runs_smw/checkpoints/SF84G_PT_38.zip
[EVAL] Mean Return: 186.300, Best Return: 186.308
New best record. Saved to ./runs_smw/best_model_A.zip
Saved video to ./runs_smw/videos/step_12451840_mean_186.30.mp4

=== Round 10 | Learn 327680 steps (Total trained: 12451840) ===
Logging to ./runs_smw/tb/SF84G_1220_20_0
---------------------------------
| time/              |          |
|    fps             | 745      |
|    iterations      | 1        |
|    time_elapsed    | 10       |
|    total_timesteps | 12460032 |
---------------------------------
---------------------------------------
| time/                 |             |
|    fps                | 634         |
|    iterations         | 2           |
|    time_elapsed       | 25          |
|    total_timesteps    | 12468224    |
| train/                |             |
|    approx_kl          | 0.027178876 |
|    entropy_loss       | -1.82       |
|    explained_variance | 0.977       |
|    learning_rate      | 0.0002

Saved checkpoint: ./runs_smw/checkpoints/SF84G_PT_39.zip
[EVAL] Mean Return: 29.276, Best Return: 29.289
Saved video to ./runs_smw/videos/step_12779520_mean_29.28.mp4

=== Round 11 | Learn 327680 steps (Total trained: 12779520) ===
Logging to ./runs_smw/tb/SF84G_1220_20_0
---------------------------------
| time/              |          |
|    fps             | 1081     |
|    iterations      | 1        |
|    time_elapsed    | 7        |
|    total_timesteps | 12787712 |
---------------------------------
---------------------------------------
| time/                 |             |
|    fps                | 852         |
|    iterations         | 2           |
|    time_elapsed       | 19          |
|    total_timesteps    | 12795904    |
| train/                |             |
|    approx_kl          | 0.018827666 |
|    entropy_loss       | -1.86       |
|    explained_variance | 0.982       |
|    learning_rate      | 0.0002      |
|    loss               | 0.0611      |
|    mean

Saved checkpoint: ./runs_smw/checkpoints/SF84G_PT_40.zip
[EVAL] Mean Return: 184.600, Best Return: 184.608
Saved video to ./runs_smw/videos/step_13107200_mean_184.60.mp4
Training finished. Environment closed.


'\ntensorboard --logdir=./runs_smw/tb\n'

## Display Video

In [ ]:
# from IPython.display import Video
# import glob

# list_of_files = glob.glob(os.path.join(VIDEO_DIR, '*.mp4')) 
# if list_of_files:
#     latest_file = max(list_of_files, key=os.path.getctime)
#     print(f"Playing: {latest_file}")
#     display(Video(latest_file, embed=True, width=600))
# else:
#     print("No videos found yet.")